# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains *(combine multiple chains, output of one chain is input of next chain)*
  * SimpleSequentialChain *(subchains with single input/output)*
  * SequentialChain *(subchains with multiple input/output)*
* Router Chain

In [2]:
import warnings
warnings.filterwarnings('ignore')

## EITHER: use your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## OR: use [LocalAI as an OpenAI replacement](https://localai.io/howtos/easy-request-openai/)

In [1]:
import os
import openai

# Specify the port your LocalAI docker container runs on
# openai.api_base = "http://localhost:8080/v1"  # default
openai.api_base = "http://localhost:9095/v1"  # for lunademo
openai.api_key = "sx-xxx"  # not needed for LocalAI (dummy)
OPENAI_API_KEY = "sx-xxx"
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [2]:
# Specify the model you are using
# llm_model = ""
# WARNING: not all LangChain functions work with custom LocalAI models, but they can be renamed, e.g. to 'gpt-3.5-turbo'
llm_model = "lunademo"  # for lunademo

## Read our `csv` data

In [5]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [4]:
df.head()



,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

**Initialize the model - we are using a high temperature (randomness) here for more varied descriptions**

In [6]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

## SimpleSequentialChain
- combine multiple chains, where output of one chain is input of next chain
- subchains with single input/output
- e.g. get the *name of company* from an individual *product description*, then get an *advertising slogan* for this company

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1 - name from product description
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [9]:
# prompt template 2 - slogan from name
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [30]:
# Example product from our csv
product = "Queen Size Sheet Set"

In [31]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Based on your description, the best name to describe the company that makes the king size set with 4 pillowcases would be "Sheets & More". The company offers a wide range of bedding products, including king size sets with 4 pillowcases, and provides excellent customer service by sending additional pillowcases when needed.
Sheets & More is a company that offers high-quality bedding products, including king size sets with 4 pillowcases, and provides excellent customer service by sending additional pillowcases when needed.

> Finished chain.


'Sheets & More is a company that offers high-quality bedding products, including king size sets with 4 pillowcases, and provides excellent customer service by sending additional pillowcases when needed.'

## SequentialChain

In [12]:
from langchain.chains import SequentialChain

**1st: Translate the review to English**

In [13]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

Note the output variable `English_Review` which is input to the next chain

**2nd: Summarize review in one sentence**

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


Note the output variable `summary` which is input to the next chain

**3rd: Get the language of the original review**

In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


Note the original input variable `Review` is used again here and output variable `summary` is input to the next chain

**4th: Now take multiple inputs**
- summary (calculated with 2nd chain)
- language (calculated with 3rd chain)

In [16]:
# prompt template 4: follow up message in specified language
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


**The overall chain (whick takes the four created chains)**

In [17]:
# overall_chain: input = Review 
# and output = English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

**Now we run the overall chain over some of the data loaded from `Data.csv`**

In [18]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "The taste is poor. The mousse does not hold, it's strange. I buy the same thing in the store and the taste is much better...",
 'summary': 'The store-bought version of the mousse is of better taste than the homemade version.',
 'followup_message': 'Réponse aux commentaires: Le version achetée en magasin de la mousse est de meilleur goût que la version à domicile faite.'}

**What happened:**
- we took a French review from data
- we translated it to English (*1st*)
- we summarised it in one sentence (*2nd*)
- we got the language of the original review (*3rd*)
- we wrote a follow up message to the summary of the review in the language detected (*4th*)
  
- **this actually worked with LocalAI and `lunademo`, a very small model (~2GB)! How cool is that?!**

## Router Chain
- if you save several subchains, each specialized to some specific type of input (e.g., related subjects)
- decides which chain to pass through

**Here we have different prompt templates, where each is good for answering questions from different fields**
- math
- physics
- history
- computer science

In [33]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

**We can provide more info about the different templates**
- name of the template
- description of the template --> this is passed to the router chain, so it can decided which template to pick

In [45]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

- **MultiPromptChain** needed for routing between multiple different prompt templates
- **LLMRouterChain** uses LLM itself to route between different subchains, uses `description` and `name` provided before
- **RouterOutputParser** parses the output into a dict, to determine which chain to use and what the input to it should be

In [46]:
from langchain.chains.router import MultiPromptChain  # needed for routing between multiple different prompt templates
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

**Import and define the language model**

In [47]:
llm = ChatOpenAI(temperature=0, model=llm_model)

**Create the destination chains**
- will be called by the router chain

In [48]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)  # each destination chain is itself an LLMchain
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

**We also need a default chain (fallback)**
- called when the router chain can't decide which subchain to use
- in our example: whether a question is more math/physics related

In [49]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

**Define the template that is used by the LLM to route between different chains**
- has instructions of the task to be done
- and specific formatting the output should be in

In [50]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

**Put it all together to build the router chain**

- full router template, formatted with the destinations defined before
- flexible to different destinations (e.g., add other subject to prompt templates and prompt infos)

In [51]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

- create the prompt template
- note the parser, which helps decide which subchains to route  between

In [52]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

- create the router chain by passing `llm` and overall `router_prompt`

In [53]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

- put together: **the overall chain**
- router chain, destination chains, default chain

In [54]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [55]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
history: {'input': 'What is the significance of the Black Death?'}
> Finished chain.


'The Black Death was a deadly pandemic that swept across Europe in the 14th century, causing the deaths of an estimated 75-200 million people. It was one of the most significant events in European history and had a profound impact on the social, economic, and political life of the continent.\nThe significance of the Black Death lies in its impact on the population, the economy, and the political system of medieval Europe. It led to a significant decrease in population, which affected the social and economic structures of society. The plague also had a significant impact on the religious and political institutions of the time, as people turned to God for answers and sought spiritual guidance during the crisis.\nFurthermore, the Black Death led to significant changes in the way people lived and worked, as well as in their attitudes towards death and the afterlife. The plague forced people to re-evaluate their priorities and values, leading to a greater emphasis on family and community ov

### Source: https://learn.deeplearning.ai/langchain/lesson/4/chains